In [1]:
import pandas as pd
import numpy as np
import pickle

from sklearn.preprocessing import LabelEncoder

import re
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import fasttext
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem import WordNetLemmatizer

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, Embedding, Bidirectional, Reshape
from tensorflow.keras.callbacks import EarlyStopping


2023-11-05 12:38:27.382033: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-05 12:38:27.398467: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-05 12:38:27.594995: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-05 12:38:27.598795: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-05 12:38:29.640909: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [2]:
dataset = '../data/Sentiment Analysis.csv'

In [3]:
df = pd.read_csv(dataset)
df.head()

,text,label
0,I always wrote this series off as being a comp...,0
1,1st watched 12/7/2002 - 3 out of 10(Dir-Steve ...,0
2,This movie was so poorly written and directed ...,0
3,The most interesting thing about Miryang (Secr...,1
4,"when i first read about ""berlin am meer"" i did...",0


In [4]:
df.drop_duplicates(inplace=True)

In [5]:
def cleanText(text):
    text = re.sub(r'RT @\w+: ', '', text)
    text = re.sub('[^a-zA-Z]', ' ', text)
    text = text.lower()
    return text    

df['text'] = df['text'].apply(cleanText)
df.head()

,text,label
0,i always wrote this series off as being a comp...,0
1,st watched out of dir steve ...,0
2,this movie was so poorly written and directed ...,0
3,the most interesting thing about miryang secr...,1
4,when i first read about berlin am meer i did...,0


In [6]:
cv = CountVectorizer()
X = cv.fit_transform(df['text'])
y = df['label']

le = LabelEncoder()
y = le.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [7]:

model = tf.keras.models.Sequential()

model.add(tf.keras.layers.Dense(32, input_dim=X_train.shape[1], activation='relu'))
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train.toarray(), y_train, epochs=10, batch_size=32, validation_data=(X_test.toarray(), y_test))

model.save('../models/sentimentDetection.h5')
pickle.dump(cv, open('../models/cvsentiment.pkl', 'wb'))

2023-11-05 12:39:11.701984: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_COMPAT_NOT_SUPPORTED_ON_DEVICE: forward compatibility was attempted on non supported HW
2023-11-05 12:39:11.702081: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:168] retrieving CUDA diagnostic information for host: n00b
2023-11-05 12:39:11.702190: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:175] hostname: n00b
2023-11-05 12:39:11.702489: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:199] libcuda reported version is: 535.129.3
2023-11-05 12:39:11.702922: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:203] kernel reported version is: 535.113.1
2023-11-05 12:39:11.702946: E tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:312] kernel version 535.113.1 does not match DSO version 535.129.3 -- cannot find working devices in this configuration


Epoch 1/10
125/125 [==============================] - 5s 28ms/step - loss: 0.4975 - accuracy: 0.7895 - val_loss: 0.3479 - val_accuracy: 0.8769
Epoch 2/10
125/125 [==============================] - 4s 30ms/step - loss: 0.1658 - accuracy: 0.9489 - val_loss: 0.3289 - val_accuracy: 0.8729
Epoch 3/10
125/125 [==============================] - 3s 25ms/step - loss: 0.0501 - accuracy: 0.9917 - val_loss: 0.3674 - val_accuracy: 0.8739
Epoch 4/10
125/125 [==============================] - 3s 26ms/step - loss: 0.0149 - accuracy: 0.9987 - val_loss: 0.3967 - val_accuracy: 0.8669
Epoch 5/10
125/125 [==============================] - 3s 26ms/step - loss: 0.0057 - accuracy: 0.9997 - val_loss: 0.4529 - val_accuracy: 0.8699
Epoch 6/10
125/125 [==============================] - 3s 25ms/step - loss: 0.0028 - accuracy: 1.0000 - val_loss: 0.4748 - val_accuracy: 0.8679
Epoch 7/10
125/125 [==============================] - 3s 27ms/step - loss: 0.0017 - accuracy: 1.0000 - val_loss: 0.4987 - val_accuracy: 0.8659